In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [8]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
# llm=ChatOpenAI(model="gpt-3.5-turbo")

In [10]:
def lengthFunction(text:str):
    return len(text)

def _multipleLengthFunction(text1:str,text2:str):
    return len(text1)*len(text2)

def multipleLengthFunction(_dict:dict):
    return _multipleLengthFunction(text1=_dict['text1'],text2=_dict['text2'])

In [11]:
prompt=ChatPromptTemplate.from_template(template="What is {a}+{b}")

In [12]:
chain=(
    {
        "a":itemgetter("foo")|RunnableLambda(func=lengthFunction),
        "b":
            {
                "text1":itemgetter("foo"),
                "text2":itemgetter("bar")
            } | RunnableLambda(func=multipleLengthFunction)
    } 
    | prompt 
    | llm 
    | StrOutputParser()
)


In [13]:
chain.invoke(input={"foo":"bar","bar":"ritz"})

'3 + 12 = 15'

<h3>Using Lambda in Streaming</h3>

In [14]:
prompt=ChatPromptTemplate.from_template(
    template="Write a comma-separated list of 5 animals similar to: {animal}. Do not include numbers or bullets, spaces or newlines"
)

In [15]:
strChain={'animal':RunnablePassthrough()} |prompt | llm |StrOutputParser()

In [16]:
strChain.invoke(input="Snake")

'Eel,Worm,LeglessLizard,Amphisbaenian,Caecilian'

In [17]:
strChain=prompt | llm |StrOutputParser()
strChain.invoke(input={"animal":"Snake"})

'Eel,Worm,LeglessLizard,Amphisbaena,Caecilian'

In [18]:
for chunk in strChain.stream(input="Snake"):
    print(chunk,flush=True,end="|")

worm|,eel,leglesslizard,slowworm,caecilian|

In [19]:
## Custom Parser that splits an iterator of llm tokens into a list of strings separated by commas
# The entire chunk is generated and passed through the Runnable Lambda

def splitIntoList(input:Iterator[str]) -> Iterator[list[str]]:
    # Hold Partial Input, until we get a Comma
    buffer=""
    for chunk in input:  # Chunks are actually the individual letters for stream
        # print(chunk)  # to be commented
        buffer+=chunk
        while "," in buffer:
            commaIndex=buffer.index(",")
            yield [buffer[:commaIndex].strip()]
            if commaIndex==len(buffer)-1:
                buffer=""
            # else:
            #     buffer=buffer[commaIndex+1:].strip()
    yield [buffer.strip()]

In [20]:
listChain={'animal':RunnablePassthrough()} |prompt | llm |StrOutputParser()|RunnableLambda(func=splitIntoList)

In [21]:
for chunk in listChain.stream(input="snake"):
    print(chunk,flush=True)

['worm']
['eel']
['leglesslizard']
['amphisbaena']
['caecilian']


In [22]:
for chunk in listChain.stream(input="wife"):
    print(chunk,flush=True)

['spouse']
['partner']
['mate']
['betterhalf']
['significantother']


#### Async Iteration

In [23]:
# Chunks wise splits are passed through each for loop 
async def asplitIntoList(input:AsyncIterator[str]) -> AsyncIterator[list[str]]:
    # Hold Partial Input, until we get a Comma
    async for (chunk) in input:  # here chunk are incremental chunks and input is the generator object
        c=chunk
        # print(c)  # comment it
        if "," not in c:
            yield [c]
        else:
            while "," in c:
                commaIndex=c.index(",")
                if c[:commaIndex].strip():
                    yield [c[:commaIndex].strip()]
                if commaIndex==len(c)-1:
                    c=""
                else:
                    c=c[commaIndex+1:].strip()
    yield [c.strip()]

In [90]:
a="snake,"
a[6:]

''

In [25]:
# Chunks wise splits are passed through each for loop 
async def asplitIntoList(
    input: AsyncIterator[str],
) -> AsyncIterator[List[str]]:  # async def
    async for (
        chunk
    ) in input:  # here chunk are incremental chunks and input is the generator object
        c=chunk
        while "," in c:
            comma_index = c.index(",")
            if c[:comma_index].strip():
                yield [c[:comma_index].strip()]
            c = c[comma_index + 1 :]
        else:
            yield [c.strip()]


In [26]:
alistChain={'animal':RunnablePassthrough()} |prompt | llm |StrOutputParser()|RunnableGenerator(transform=asplitIntoList)

In [27]:
async for chunk in alistChain.astream(input="tiger"):
    print(chunk,flush=True)

['lion']
['leopard']
['jaguar']
['cheetah']
['snowleopard']
